<a href="https://colab.research.google.com/github/vishwanaik15/DM-ASSIGNMENT03/blob/main/Assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Vishwa Chetankumar Naik</br>
UTA ID:1001871311</br>
Assignment - 03 </br>


**Import Libraries**

In [108]:
import os
import glob
import numpy as np
import pandas as pd
from pandas import Series
import re
from sys import path
from collections import Counter
import string
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re


**A)Divide the dataset as train and development.**</br>
--Here I have used dataset from kaggle sentiment labelled sentences dataset.</br>
--I have imported test data at the last of  assignment because in the last question it is required, so just to remove confusion.




In [109]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [110]:
imdb_data = pd.read_csv('/content/gdrive/My Drive/data/imdb_labelled.txt', delimiter='\t', header=None, names=['review', 'sentiment'])
print(imdb_data.shape)

(748, 2)


#dataset understanding and preprocess

In [111]:
imdb_data.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [112]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [113]:
imdb_data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [114]:
imdb_data.sentiment.value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [115]:
shuffle = imdb_data.sample(frac=1).to_numpy()
#shuffling data using sample method to get random values in train and development set each time 
size = int(0.65*len(imdb_data))
#i have taken 65% of data as training and rest of 40% data as development set 
train_set = shuffle[:size]
development_set = shuffle[size:]
print("Size of training set: ",len(train_set))
print("Shape of training set:",{train_set.shape})
print("Size of develpment set: ",len(development_set))
print("Shape of development set:",{development_set.shape})


Size of training set:  486
Shape of training set: {(486, 2)}
Size of develpment set:  262
Shape of development set: {(262, 2)}


In [116]:
X_train = train_set[:,0] #reviews in X_train
Y_train = train_set[:,-1] #last column goes in Y_train which shows labels 0 or 1
X_development = development_set[:,0]
Y_development = development_set[:,-1]

**b>Build a vocabulary as list. **</br>
[‘the’ ‘I’ ‘happy’ … ] </br>
You may omit rare words for example if the occurrence is less than five times
A reverse index as the key value might be handy</br>
{“the”: 0, “I”:1, “happy”:2 , … }


In [117]:
#vocabulary list
a = {}
vocab_list=[]
def build_vocab_list(imdb_data):
    a = counting_number_of_unique_words(imdb_data)
    for i in a.keys():
        vocab_list.append(i)
    return vocab_list

In [118]:
#counting unique number of words in dataset so we can compare after removing less occurring words 
words_count = {}
def counting_number_of_unique_words(imdb_data):
    for line in imdb_data:
        for word in line.split():
            if word in words_count:
                words_count[word] = words_count[word] + 1
            else:
                words_count[word] = 1
    return words_count


In [119]:
#before removing less occurring data 
words_count = counting_number_of_unique_words(X_train)
print("Total unique words in the given dataset before removing less occurring words: ",len(words_count))


Total unique words in the given dataset before removing less occurring words:  2756


In [120]:
print(words_count) #with less occurring words/


{'However': 1, 'Paul': 3, 'Schrader': 1, 'has': 25, 'indeed': 1, 'made': 12, 'a': 233, 'film': 58, '"about"': 1, 'Mishima': 1, 'that': 87, 'is': 182, 'both': 7, 'superb': 2, '&': 7, 'complex.': 1, 'The': 107, 'rest': 3, 'of': 199, 'the': 366, 'movie': 70, 'lacks': 4, 'art,': 2, 'charm,': 1, 'meaning...': 1, 'If': 7, "it's": 17, 'about': 25, 'emptiness,': 1, 'it': 84, 'works': 3, 'I': 142, 'guess': 3, 'because': 12, 'empty.': 1, 'All': 6, 'actors': 7, 'give': 5, 'wonderful': 9, 'performance,': 1, 'especially': 4, 'Jennifer': 1, 'Rubin': 1, 'as': 52, 'Jamie': 1, 'Harris,': 1, 'who': 23, 'changes': 1, 'from': 24, 'nervous': 1, 'starlet': 1, 'in': 116, 'beginning': 3, 'through': 5, 'strange': 1, 'events': 1, 'she': 5, 'part': 6, 'to': 128, 'cool': 4, 'star.': 1, 'But': 8, 'if': 16, 'you': 31, 'liked': 6, 'movies': 12, 'like': 25, 'Matrix': 1, '(and': 3, 'better': 8, 'yet,': 1, 'their': 10, 'sequels)': 1, 'think': 5, "you'll": 1, 'appreciate': 4, 'thought': 6, 'provoking,': 1, 'mindblowing'

In [121]:
#cleandata
#reomving less frequent words from dataset
repeating_words = []
for i in words_count.keys():
    if (words_count[i]<5):
          repeating_words.append(i)
for i in repeating_words:
    del words_count[i]
len(words_count)
def removing_less_occurring_words(imdb_data,words_count):
    for i, review in enumerate(imdb_data):
        clean = []
        for word in review.split():
            if word in words_count.keys():
                clean.append(word)
        clean = ' '.join(clean)
        imdb_data[i] = clean
    return imdb_data

In [122]:
#after removing less occurring data from vocab_list and so from X_train
X_train = removing_less_occurring_words(X_train,words_count)
print("Total unique words in the given dataset after removing less occurring words: ",len(X_train))
print('')
#print("Length of words dictionary after removing less occurred data: ",len(words_count))

Total unique words in the given dataset after removing less occurring words:  486



In [123]:
#my words dictionary after removing less occurring words
print(words_count)


{'has': 25, 'made': 12, 'a': 233, 'film': 58, 'that': 87, 'is': 182, 'both': 7, '&': 7, 'The': 107, 'of': 199, 'the': 366, 'movie': 70, 'If': 7, "it's": 17, 'about': 25, 'it': 84, 'I': 142, 'because': 12, 'All': 6, 'actors': 7, 'give': 5, 'wonderful': 9, 'as': 52, 'who': 23, 'from': 24, 'in': 116, 'through': 5, 'she': 5, 'part': 6, 'to': 128, 'But': 8, 'if': 16, 'you': 31, 'liked': 6, 'movies': 12, 'like': 25, 'better': 8, 'their': 10, 'think': 5, 'thought': 6, 'this': 106, 'will': 14, 'watch': 6, 'This': 44, 'and': 235, 'between': 7, 'we': 6, 'saw': 7, 'with': 50, 'off': 5, 'was': 104, 'how': 15, 'little': 12, 'there': 14, 'on': 35, 'two': 12, 'Not': 5, 'for': 52, "don't": 10, 'anything': 5, 'or': 25, 'even': 17, 'its': 16, 'were': 17, 'just': 35, 'recommend': 8, 'it.': 20, 'one': 26, 'any': 15, 'real': 8, 'There': 6, 'film,': 6, 'understand': 6, 'he': 18, 'would': 8, 'his': 21, 'out': 19, 'really': 25, 'loved': 5, 'story': 13, 'line': 5, 'anyone': 7, 'an': 26, 'watching': 11, 'her': 

In [124]:
review_0_train = train_set[Y_train == 0]
print(f'Total 0 (negative) review in train dataset : {len(review_0_train)}')

review_1_train = train_set[Y_train == 1]
print(f'Total 1 (positive) review in train dataset : {len(review_1_train)}')

Total 0 (negative) review in train dataset : 230
Total 1 (positive) review in train dataset : 256


c.) Find basic probability gor positive and negative.

In [125]:
#count basic probability for positive and negative reviews
def calculate_prior_class_prob(Y_train):
  prior_class_prob = {}
  for c in np.unique(Y_train):
    count = sum(Y_train == c)
    prior_class_prob[c] = count / Y_train.size
  return prior_class_prob

prior_class_prob = calculate_prior_class_prob(Y_train)
print(f'prior probability of class 0 (negative) : {prior_class_prob[0]}')
print(f'prior probability of class 1 (positive) : {prior_class_prob[1]}')

prior probability of class 0 (negative) : 0.4732510288065844
prior probability of class 1 (positive) : 0.5267489711934157


In [126]:
#here we are making dictionary containing word and it's occurrance in data
def total_words_in_data(imdb_data, words_count):
    new_dict = {}
    for word in words_count.keys():
        count = 0
        for review in imdb_data:
            if word in review:
                count = count + 1
        new_dict[word] = count
    return new_dict

In [127]:
words_occ = total_words_in_data(X_train,words_count)
print("Number of word the in the data : ",words_occ['the'])

Number of word the in the data :  238


**-->Probability of the occurrence**</br>
P[“the”] = num of documents containing ‘the’ / num of all documents


In [128]:
#probability of given word in dataset
def prob_calc_of_word_in_data(imdb_data, word):
    count = 0
    if word not in words_occ.keys():
        count = 0
    else:
        count = words_occ[word]
    return count/len(imdb_data)

In [129]:
prob_word_in_data = prob_calc_of_word_in_data(X_train, 'the')
print("Probability of word in whole document: p[the] %: ",prob_word_in_data*100)

Probability of word in whole document: p[the] %:  48.971193415637856


**-->Conditional probability based on the sentiment</br>**
P[“the” | Positive]  = # of positive documents containing “the”  


In [130]:
def positive_words_count(imdb_data, label, words_count):
    positive_words = {}
    for word in words_count.keys():
        pos_count = 0
        for index, review in enumerate(imdb_data):
            if word in review.split() and label[index]==1:
                pos_count = pos_count + 1
        positive_words[word] = pos_count
    return positive_words

def negative_words_count(imdb_data, label, words_count):
    negative_words = {}
    for word in words_count.keys():
        neg_count = 0
        for index, review in enumerate(imdb_data):
            if word in review.split() and label[index]==0:
                neg_count = neg_count + 1
        negative_words[word] = neg_count
    return negative_words
positive_words = positive_words_count(X_train, Y_train, words_count)
negative_words = negative_words_count(X_train, Y_train, words_count)

In [131]:
#in how many positive documents word "the" comes
positive_words['the']

126

In [132]:
#in how many negative documents word "the" comes
negative_words['the']

92

In [133]:
def positive_doc_count(imdb_data):
    positive_doc = 0
    for i in range(len(imdb_data)):
        if (imdb_data[i]==1):
            positive_doc = positive_doc + 1
    return positive_doc
def negative_doc_count(imdb_data):
    negative_doc = 0
    for i in range(len(imdb_data)):
        if (imdb_data[i]==0):
            negative_doc = negative_doc + 1
    return negative_doc
def pos_neg_doc_count(imdb_data, attribute):
    if attribute == "positive":
        cnt = positive_doc_count(imdb_data)
    else:
        cnt = negative_doc_count(imdb_data)
    return cnt
positive_doc = pos_neg_doc_count(Y_train,attribute = "positive")
negative_doc = pos_neg_doc_count(Y_train,attribute = "negative")

In [134]:
#number of positive documents
print(positive_doc)
print(negative_doc)

256
230


In [135]:
#probability of given word in positive or negative document
def num_of_word_in_posneg(word,words_count, attribute):
    count = 0
    if attribute =="positive":
        if word in positive_words.keys():
            count = positive_words[word]
        else:
            count = 0
            
    elif attribute=="negative":
        if word in negative_words.keys():
            count = negative_words[word]
        else:
            count = 0
    return count

In [136]:
numerator = num_of_word_in_posneg('the',words_count, attribute = "positive")
p = numerator/positive_doc
print("p[the|positive]: "+str(p*100)+" %")

p[the|positive]: 49.21875 %


In [137]:
numerator = num_of_word_in_posneg('the',words_count, attribute = "negative")
p = numerator/positive_doc
print("p[the|negative]: "+str(p*100)+" %")

p[the|negative]: 35.9375 %


In [138]:
#probability for p[positive|the] without NAIVE BAYES
prob = (p*(positive_doc/len(X_train)))/prob_word_in_data
print("Probability of given word in data",prob*100)

Probability of given word in data 38.655462184873954


**D) Calculate accuracy using dev dataset **


In [139]:
def NaiveBayes(imdb_data, label, word,words_count, attribute):
    numerator = num_of_word_in_posneg(word, words_count,attribute)
    denominator = prob_calc_of_word_in_data(imdb_data, word)
    posnegdoc = pos_neg_doc_count(label,attribute)
    if denominator == 0:
        denominator = 0.1
    else:
        denominator = denominator
    result = (numerator/posnegdoc)*(posnegdoc/len(imdb_data))/denominator
    return result

In [140]:
def NaiveBayes_for_data(imdb_data,label,words_count,data1):
    positive_pred = 1
    negative_pred = 1
    y_pred = []
    for index,review in enumerate(data1):
        for word in review.split(" "):
            a = NaiveBayes(imdb_data, label, word,words_count, attribute = "negative")
            if a == 0:
                a = 0.1
            else:
                a = a
            negative_pred = negative_pred * a
            #print("n",negative_pred)
            b = NaiveBayes(imdb_data, label, word,words_count, attribute = "positive")
            if b == 0:
                b = 0.1
            else:
                b = b
            positive_pred = positive_pred * b
            #print("p",positive_pred)
        if positive_pred > negative_pred:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [141]:
y_pred = NaiveBayes_for_data(X_train,Y_train,words_count,X_development)

In [142]:
def accuracy(label, prediction):
    pred = 0
    for i in range(len(label)):
        if(label[i]==prediction[i]):
            pred = pred + 1
        acc = float(pred/len(label))
    return acc

 **E)Do following experiments
Compare the effect of Smoothing**



In [143]:
#Accuracy without smoothing
Accuracy = accuracy(Y_development,y_pred)
print("Accuracy without smoothing the data:  "+str(Accuracy*100)+" %")

Accuracy without smoothing the data:  47.70992366412214 %


In [144]:
#smoothing the data
def NaiveBayesSmooth(imdb_data, label, word,words_count, attribute):
    numerator = num_of_word_in_posneg(word, words_count, attribute)
    denominator = prob_calc_of_word_in_data(imdb_data, word)
    posnegdoc = pos_neg_doc_count(label,attribute)
    if denominator == 0:
        denominator = 0.1
    else:
        denominator = denominator
    result = (((numerator/posnegdoc)*(posnegdoc/len(imdb_data)))+1)/(denominator+2)
    return result
def NaiveBayes_for_Smoothdata(imdb_data,label,words_count,data1):
    positive_pred = 1
    negative_pred = 1
    y_pred = []
    for index,review in enumerate(data1):
        for word in review.split(" "):
            a = NaiveBayesSmooth(imdb_data, label, word,words_count, attribute = "negative")
            if a == 0:
                a = 0.1
            else:
                a = a
            negative_pred = negative_pred * a
            #print("n",negative_pred)
            b = NaiveBayesSmooth(imdb_data, label, word,words_count, attribute = "positive")
            if b == 0:
                b = 0.1
            else:
                b = b
            positive_pred = positive_pred * b
            #print("p",positive_pred)
        if positive_pred > negative_pred:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [145]:
#predicted values for given data 
y_pred = NaiveBayes_for_Smoothdata(X_train,Y_train,words_count,X_development)

In [146]:
#accuracy after smoothing the data
Accuracy = accuracy(Y_development,y_pred)
print("Accuracy after smoothing the data:  "+str(Accuracy*100)+" %")

Accuracy after smoothing the data:  50.76335877862596 %


**-->Derive Top 10 words that predicts positive and negative class
P[Positive| word]**

In [147]:
import operator,itertools
top_positive={}
for word in words_count.keys():
    acc = NaiveBayes(X_train, Y_train, word,words_count, attribute="positive")
    top_positive[word] = acc
top_reversed = dict(sorted(top_positive.items(),key=operator.itemgetter(1),reverse=True))
top_10_positive = dict(itertools.islice(top_reversed.items(), 10))

In [148]:
#top10 positive 
print("Top 10 words that predict positive class\n",top_10_positive.keys())

Top 10 words that predict positive class
 dict_keys(['wonderful', 'liked', 'think', 'loved', 'watch.', 'well.', 'makes', 'funny', 'years.', 'interesting'])


In [149]:
top_negative={}
for word in words_count.keys():
    acc = NaiveBayes(X_train, Y_train, word,words_count, attribute="negative")
    top_negative[word] = acc
top_reversed = dict(sorted(top_positive.items(),key=operator.itemgetter(1),reverse=False))
top_10_negative = dict(itertools.islice(top_reversed.items(), 10))

In [150]:
top_negative={}
for word in words_count.keys():
    acc = NaiveBayes(X_train, Y_train, word,words_count, attribute="negative")
    top_negative[word] = acc
top_reversed = dict(sorted(top_positive.items(),key=operator.itemgetter(1),reverse=False))
top_10_negative = dict(itertools.islice(top_reversed.items(), 10))

**F)Using the test dataset
Use the optimal hyperparameters you found in the step e, and use it to calculate the final accuracy.**

In [151]:
shuffle = imdb_data.sample(frac=1).to_numpy()
#shuffling data using sample method to get random values in train and development set each time 
size = int(0.35*len(imdb_data))
#i have taken 35% of data as testing.
test_set = shuffle[:size]

print("Size of test set: ",len(test_set))
print("Shape of test set:",{test_set.shape})



Size of test set:  261
Shape of test set: {(261, 2)}


In [152]:
X_test = test_set[:,0] #reviews in X_train
Y_test = test_set[:,-1]

In [153]:
y_pred = NaiveBayes_for_data(X_train,Y_train,words_count,X_test)

In [154]:
#accuracy without smoothing the test data
Accuracy = accuracy(Y_test,y_pred)
print("Accuracy without smoothing the TEST data:  "+str(Accuracy*100)+" %")

Accuracy without smoothing the TEST data:  51.34099616858238 %


In [155]:
y_pred = NaiveBayes_for_Smoothdata(X_train,Y_train,words_count,X_test)

In [156]:
#accuracy after smoothing the test data
Accuracy = accuracy(Y_test,y_pred)
print("Accuracy after smoothing the TEST data:  "+str(Accuracy*100)+" %")

Accuracy after smoothing the TEST data:  53.63984674329502 %
